In [1]:
#from ipynb.fs.full.LocationCleaningScript import clean_location
from ipynb.fs.full.cleanIndustries import cleanIndustries
# from ipynb.fs.full.CurrencyConverter import *
#from ipynb.fs.full.NormalizeWorkExperience import NormalizeExperience
import pandas as pd
import urllib.request
import json
from datascience import *
import numpy as np
import re
import math
def RemoveSpacesFromTitles(df):
    
    df.columns = [c.replace(' ', '_') for c in df.columns]
    return df
def NormalizeExperience(filename):
    df = pd.read_csv(filename)
    df = df.replace(to_replace = ['5-7 years'], value = '5 - 7 years')
    df = df.replace(to_replace = ['1 year or less'], value = '0 - 1 years')
    return df
  

#x = x.to_df()


In [2]:
class currencyConverter:
    rates = {}
    def __init__(self, url):
        data = urllib.request.urlopen(url).read().decode('utf-8')
        f = json.loads(data)
        self.rates = f["rates"]
        
        
    def convert(self, amount, f, t):
        output = 0
        if f != 'USD':
            output = amount / self.rates[f] * self.rates['USD']
            return output
        return amount
        
def convertCurrency(df):
    for i in range(len(df['Currency'])):
        df["Annual_Salary"][i]= str(    df["Annual_Salary"][i] )
        df["Annual_Salary"][i]= ''.join(i for i in df["Annual_Salary"][i] if i.isdigit())
        if df["Currency"][i] == "Other" or df["Annual_Salary"][i] == "":
            df["Annual_Salary"][i] = 0
        else:
            df["Annual_Salary"][i]=float(    df["Annual_Salary"][i] )
        
       

        
        if df["Currency"][i] in c.rates and df["Currency"][i] != "USD" :
            df['Annual_Salary'][i] =c.convert(float(df['Annual_Salary'][i]), df["Currency"][i], 'USD')

    return df

c = currencyConverter("http://data.fixer.io/api/latest?access_key=238f1925907e0f01a7fa1dc2b4042967")

In [3]:
def clean2019locs(tbl):
    data = list(tbl["Location"])
    for i in range(len(data)):
        data[i] = str(data[i])
        data[i] = data[i].strip()
        chars = ["'",'"',',','.','/',"-",'—']
        if '(' in data[i]:
            data[i] =data[i][:data[i].find("(")]
        for c in range(len(chars)):
            if chars[c] in data[i]:
                data[i] = data[i].replace(chars[c]," ")
    tbl["Location"] = data
    return tbl
        
        

In [8]:
table = NormalizeExperience('../Data/2019_Results.csv')
table = RemoveSpacesFromTitles(table)
table = cleanIndustries(table)
table = convertCurrency(table)
table = clean2019locs(table)
table = Table.from_df(table)
table = table.where("Annual_Salary",are.below(10000000))
#print(np.unique(table["Currency"]))
#table.to_csv("CleanedData2019.csv")


Age,Industry,Job_Title,Annual_Salary,Currency,Location,Work_Experience,Context_for_Job,Other
55-64,Other,Systems Analyst,9.9913e+06,USD,Jacksonville Florida USA,31 - 40 years,nan,nan
65 or over,Government and Public Administration,Engineering Manager,9.9437e+06,USD,Durham NC USA,31 - 40 years,nan,nan
45-54,Art & Design,Senior Project Accountant,9.9e+06,USD,New York NY,21 - 30 years,nan,nan
55-64,Health care,Accountant,9.8e+06,USD,San Diego CA USA,31 - 40 years,nan,nan
35-44,Health care,Senior clinical analyst,9.8e+06,USD,Winston salem nc United States,11 - 20 years,nan,nan
45-54,Law or Law Enforcement,executive assistant,9.775e+06,USD,Palo Alto California USA,11 - 20 years,Executive Administrative Assistant - no legal filings/st ...,nan
35-44,Research,Business Development Manager,9.6039e+06,USD,New York NY,11 - 20 years,nan,nan
55-64,Other,Director of Early Intervention,9.6e+06,USD,Los Angeles CA,21 - 30 years,nan,nan
25-34,Insurance,Claim Manager,9.5226e+06,USD,Saint Louis MO,8 - 10 years,nan,nan
35-44,Other,"Manager, Financial Services",9.50004e+06,USD,Cleveland OH 44114,8 - 10 years,nan,nan
